<a href="https://colab.research.google.com/github/samuelguedes/AcoesDescontadasPrecoBazinGordon/blob/main/AcoesDescontadasPrecoBazinGordon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai
!pip install -q -U yfinance

Importações

In [4]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt
from scipy.stats import trim_mean
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
import google.generativeai as genai
from google.colab import userdata
from google.colab import drive

drive.mount('/content/drive')

GOOGLE_API_KEY=userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Mounted at /content/drive


Coletar os dividendos por ano

In [5]:
def coletar_dividendos_por_ano(simbolo_acao):
  dados_acao = yf.Ticker(simbolo_acao)
  historico_dividendos = dados_acao.dividends.reset_index()
  if historico_dividendos.empty:
    return None

  historico_dividendos['Year'] = historico_dividendos['Date'].dt.year
  dividendos_por_ano = historico_dividendos.groupby('Year')['Dividends'].sum()
  return dividendos_por_ano

Calcular o dividendo médio

In [6]:
def calcular_dividendo_medio(dividendos_por_ano, qtd_anos=5):
  ano_corrente = dt.datetime.now().year
  anos_completos = dividendos_por_ano[dividendos_por_ano.index < ano_corrente]
  return anos_completos.tail(qtd_anos).mean()

Calcular o preço justo Bazin

In [7]:
def calcular_preco_bazin(dividendo, taxa_retorno=0.06):
  return dividendo / taxa_retorno

Calcular o preço justo Gordon

In [8]:
def calcular_preco_gordon(dividendo, taxa_retorno=0.06, taxa_crescimento=0.005):
  return dividendo / (taxa_retorno - taxa_crescimento)

Comparação dos ativos do indice de dividendos IDIV B3

In [9]:
documento = pd.read_csv('/content/drive/MyDrive/acao/IDIVQuad_5-2024.csv', sep=';', encoding='latin-1')
df = pd.DataFrame(documento)
df.columns = ['codigo', 'acao', 'tipo', 'qtde_teorica', 'part']
df

,codigo,acao,tipo,qtde_teorica,part
0,BRSR6,BANRISUL,PNB ED N1,301.212.485,"0,646"
1,BBSE3,BBSEGURIDADE,ON NM,644.598.942,"4,113"
2,BBDC3,BRADESCO,ON EJ N1,1.095.602.044,"2,522"
3,BBDC4,BRADESCO,PN EJ N1,928.931.988,"2,392"
4,BRAP4,BRADESPAR,PN N1,373.276.882,"1,445"
5,BBAS3,BRASIL,ON NM,704.206.631,"3,746"
6,AGRO3,BRASILAGRO,ON NM,94.906.642,"0,465"
7,CXSE3,CAIXA SEGURI,ON NM,769.626.747,"2,383"
8,CMIG3,CEMIG,ON N1,697.501.384,"1,633"
9,CMIG4,CEMIG,PN N1,2.779.044.864,"5,422"


In [10]:

series_simbolos_acoes = df['codigo'].apply(lambda x: x + '.SA')
simbolos_acoes = series_simbolos_acoes.to_numpy()
simbolos_acoes

array(['BRSR6.SA', 'BBSE3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BRAP4.SA',
       'BBAS3.SA', 'AGRO3.SA', 'CXSE3.SA', 'CMIG3.SA', 'CMIG4.SA',
       'CIEL3.SA', 'CSMG3.SA', 'CPLE3.SA', 'CPLE6.SA', 'CPFE3.SA',
       'CMIN3.SA', 'CURY3.SA', 'DIRR3.SA', 'EGIE3.SA', 'FESA4.SA',
       'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'RANI3.SA', 'ITSA4.SA',
       'JBSS3.SA', 'JHSF3.SA', 'KEPL3.SA', 'KLBN11.SA', 'LAVV3.SA',
       'POMO4.SA', 'LEVE3.SA', 'BEEF3.SA', 'MTRE3.SA', 'PETR3.SA',
       'PETR4.SA', 'PLPL3.SA', 'POSI3.SA', 'SAPR4.SA', 'SANB11.SA',
       'STBP3.SA', 'CSNA3.SA', 'TAEE11.SA', 'TASA4.SA', 'TGMA3.SA',
       'VIVT3.SA', 'TIMS3.SA', 'TRPL4.SA', 'UNIP6.SA', 'USIM5.SA',
       'VALE3.SA', 'WIZC3.SA'], dtype=object)

Calcular preço justo de muitas ações

In [11]:
def calcular_precos_justos(simbolos_acoes, anos=5):
  data = []
  for simbolo_acao in simbolos_acoes:
    acao = yf.Ticker(simbolo_acao)

    dividendos = coletar_dividendos_por_ano(simbolo_acao)
    if dividendos is not None:
      media_div = calcular_dividendo_medio(dividendos, anos)
      preco_justo_gordon = calcular_preco_gordon(media_div, taxa_retorno, taxa_crescimento)
      preco_justo_bazin = calcular_preco_bazin(media_div, taxa_retorno)
      preco_atual = acao.info['currentPrice']

      diferenca_gordon = (preco_atual / preco_justo_gordon - 1) * 100
      diferenca_bazin = (preco_atual / preco_justo_bazin - 1) * 100

      data.append([acao.info['symbol'], preco_atual, preco_justo_gordon, diferenca_gordon, preco_justo_bazin, diferenca_bazin])
    else:
      printf(f"Não foi possível obter dados históricos para {simbolo_acao}.")

  df = pd.DataFrame(data, columns=['acao', 'Preço Atual', 'Preço Justo Gordon', 'Diferença Gordon (%)', 'Preço Justo Bazin', 'Diferença Bazin (%)'])
  df = df.sort_values('Diferença Gordon (%)', ascending=True)
  return df

In [59]:
taxa_retorno = 0.06
taxa_crescimento = 0.01
anos = 5
precos_justos_df = calcular_precos_justos(simbolos_acoes)
precos_justos_df

,Ação,Preço Atual,Preço Justo Gordon,Diferença Gordon (%),Preço Justo Bazin,Diferença Bazin (%)
4,BRAP4.SA,20.10,79.586332,-74.744407,66.321943,-69.693289
35,PETR4.SA,41.58,116.107016,-64.188211,96.755847,-57.025853
34,PETR3.SA,44.13,121.956060,-63.814836,101.630050,-56.577804
15,CMIN3.SA,5.07,13.913933,-63.561706,11.594944,-56.274047
48,UNIP6.SA,52.00,115.486472,-54.973081,96.238727,-45.967697
43,TASA4.SA,11.28,23.555416,-52.112924,19.629513,-42.535509
33,MTRE3.SA,3.94,8.116100,-51.454516,6.763417,-41.745420
51,WIZC3.SA,6.72,13.689016,-50.909547,11.407513,-41.091456
6,AGRO3.SA,25.43,50.909976,-50.049083,42.424980,-40.058899
22,GOAU4.SA,11.10,21.960000,-49.453552,18.300000,-39.344262


In [98]:
model = "gemini-1.0-pro"
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}
prompt = f"A ação com a Diferença Gordon (%) menor é a com valor -74.744407. Com base em {precos_justos_df}. Qual a ação com menor Diferença Gordon (%)?"

gai = genai.GenerativeModel(model, generation_config=generation_config)
response = gai.generate_content(prompt)
print(response.text)

BRAP4.SA
